In [53]:
class Variable:
    data: float
    label: str
    _children: tuple
    _grad: float
    _local_derivative: callable

    def __init__(self, val, label=None, children=()):
        self.data = val
        self.label = label
        self._grad = 0
        self._children = children
        self._local_derivative = lambda: 1
    
    def __repr__(self):
        return f"Variable({self.data})"
    
    def backwards(self):
        self._grad = 1
        self._backprop()

    def _backprop(self):
        for child in self._children:
            child._grad += self._grad * child._local_derivative()
            child._backprop()
    
    def clear_grad(self):
        self._grad = 0
        for child in self._children:
            child.clear_grad()
    
    @property
    def grad(self):
        return self._grad

    def __add__(self, other):
        self._local_derivative = lambda: 1
        other._local_derivative = lambda: 1
        return Variable(self.data + other.data, f"({self.label}+{other.label})", (self, other))
    
    def __sub__(self, other):
        self._local_derivative = lambda: 1
        other._local_derivative = lambda: -1
        return Variable(self.data - other.data, f"({self.label}-{other.label})", (self, other))

    def __mul__(self, other):
        self._local_derivative = lambda: other.data
        other._local_derivative = lambda: self.data
        return Variable(self.data * other.data, f"({self.label}*{other.label})", (self, other))

In [50]:
a = Variable(1, 'a')
b = Variable(2, 'b')
c = Variable(3, 'c')
d = a+b
e = b-c
y = d*e

print(y)

y.backwards()
print(a.grad, b.grad, c.grad)

Variable(-3)
-1 2 -3
